In [1]:

import pickle
import numpy
from music21 import note, chord

from keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate
from keras.models import Model
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

from utils import get_distinct, create_lookups, prepare_sequences, get_music_list

/Users/davidfoster/.virtualenvs/gdl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# notes

In [14]:
music_type = 'local'
mode = 'build'

if mode == 'build':
    music_list, parser = get_music_list(music_type)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        intervals = range(-6,5,1)

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.append('START')
            durations.append(0)

            for element in score.flat:
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

            notes.append('END')
            durations.append(0)

    with open('data/notes', 'wb') as filepath:
            pickle.dump(notes, filepath) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open('data/durations', 'wb') as filepath:
        pickle.dump(durations, filepath) 
else:
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open('data/durations', 'rb') as filepath:
        durations = pickle.load(filepath) 

35 files in total
1 Parsing midi_songs/cs1-2all.mid
2 Parsing midi_songs/cs5-1pre.mid
3 Parsing midi_songs/cs4-1pre.mid
4 Parsing midi_songs/cs3-5bou.mid
5 Parsing midi_songs/cs1-4sar.mid
6 Parsing midi_songs/cs2-5men.mid
7 Parsing midi_songs/cs3-3cou.mid
8 Parsing midi_songs/cs2-3cou.mid
9 Parsing midi_songs/cs1-6gig.mid
10 Parsing midi_songs/cs6-4sar.mid
11 Parsing midi_songs/cs4-5bou.mid
12 Parsing midi_songs/cs4-3cou.mid
13 Parsing midi_songs/cs5-3cou.mid
14 Parsing midi_songs/cs6-5gav.mid
15 Parsing midi_songs/cs6-6gig.mid
16 Parsing midi_songs/cs2-1pre.mid
17 Parsing midi_songs/cs3-1pre.mid
18 Parsing midi_songs/cs3-6gig.mid
19 Parsing midi_songs/cs2-6gig.mid
20 Parsing midi_songs/cs2-4sar.mid
21 Parsing midi_songs/cs3-4sar.mid
22 Parsing midi_songs/cs1-5men.mid
23 Parsing midi_songs/cs1-3cou.mid
24 Parsing midi_songs/cs6-1pre.mid
25 Parsing midi_songs/cs2-2all.mid
26 Parsing midi_songs/cs3-2all.mid
27 Parsing midi_songs/cs1-1pre.mid
28 Parsing midi_songs/cs5-2all.mid
29 Parsing 

## Create the lookup tables

In [15]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open('data/distincts', 'wb') as filepath:
    pickle.dump(distincts, filepath)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open('data/lookups', 'wb') as filepath:
    pickle.dump(lookups, filepath)

## Prepare the sequences used by the Neural Network

In [16]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts)

## create the structure of the neural network

In [19]:
seq_len = None
embed_size = 100

notes_in = Input(shape = (seq_len,))
durations_in = Input(shape = (seq_len,))

x1 = Embedding(n_notes, embed_size)(notes_in)
x2 = Embedding(n_durations, embed_size)(durations_in) 

x = Concatenate()([x1,x2])

x = LSTM(256, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = LSTM(256)(x)
x = Dropout(0.2)(x)
notes_out = Dense(n_notes, activation = 'softmax', name = 'notes')(x)
durations_out = Dense(n_durations, activation = 'softmax', name = 'durations')(x)

model = Model([notes_in, durations_in], [notes_out, durations_out])

model.summary()

opti = RMSprop(lr = 0.001)
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy']
              , loss_weights = [5, 1]
              , optimizer=opti
              )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 100)    185200      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 100)    1800        input_8[0][0]                    
__________________________________________________________________________________________________
concatenat

## Train the neural network

In [20]:

checkpoint1 = ModelCheckpoint(
    "./weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5",
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    "weights.h5",
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights('./weights/weights.h5')
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )



Train on 237160 samples, validate on 59291 samples
Epoch 1/2000000
237160/237160 [==============================] - 996s 4ms/step - loss: 16.3442 - notes_loss: 3.1557 - durations_loss: 0.5659 - val_loss: 14.1969 - val_notes_loss: 2.7227 - val_durations_loss: 0.5832
Epoch 2/2000000
237160/237160 [==============================] - 1040s 4ms/step - loss: 14.1865 - notes_loss: 2.7458 - durations_loss: 0.4573 - val_loss: 14.5362 - val_notes_loss: 2.7850 - val_durations_loss: 0.6110
Epoch 3/2000000
237160/237160 [==============================] - 1111s 5ms/step - loss: 14.1997 - notes_loss: 2.7548 - durations_loss: 0.4256 - val_loss: 15.0608 - val_notes_loss: 2.8849 - val_durations_loss: 0.6364
Epoch 4/2000000
237160/237160 [==============================] - 1107s 5ms/step - loss: 14.5289 - notes_loss: 2.8248 - durations_loss: 0.4049 - val_loss: 15.6651 - val_notes_loss: 2.9983 - val_durations_loss: 0.6738
Epoch 5/2000000
237160/237160 [==============================] - 1120s 5ms/step - loss

KeyboardInterrupt: 